# Basin Scale Papuan Fold & Thrust Belt Models

In [1]:
import UWGeodynamics as GEO
import glucifer
import numpy as np

loaded rc file /usr/local/lib/python3.5/dist-packages/UWGeodynamics/uwgeo-data/uwgeodynamicsrc


In [2]:
print("UWGeo version:", GEO.__version__)

UWGeo version: 2.8.5


In [3]:
u = GEO.UnitRegistry

## Scaling

In [4]:
half_rate = 10 * u.millimeter / u.year
model_length = 42e3 * u.meter
surfaceTemp = 273.15 * u.degK
baseModelTemp = 1603.15 * u.degK
bodyforce = 2700 * u.kilogram / u.metre**3 * 9.81 * u.meter / u.second**2
KL = model_length
Kt = KL / half_rate
KM = bodyforce * KL**2 * Kt**2
KT = (baseModelTemp - surfaceTemp)
GEO.scaling_coefficients["[length]"] = KL
GEO.scaling_coefficients["[time]"] = Kt
GEO.scaling_coefficients["[mass]"]= KM
GEO.scaling_coefficients["[temperature]"] = KT

## Model Setup

In [5]:
# Optional numerical resolution

res = (256, 128) # for running in binder. Considered under resolved but quick to run/test.
#res = (512, 256) # ~80m resolution, consider running in parallel
#(525, 217), #80m resolution - JoeI's original resolution

Model = GEO.Model(elementRes=res,
                  minCoord=(-21. * u.kilometer, -11.6 * u.kilometer),
                  maxCoord=(21. * u.kilometer, 6.4 * u.kilometer),
                  gravity=(0.0, -9.81 * u.meter / u.second**2))

In [6]:
Model.outputDir="PNG_Model"

For all rocks, we assume a heat capacity of 1000 J kg−1 K−1 and thermal diffusivity of 9·10−7 m2 s−1. The crustal thermal conductivity is therefore 2.45 W m−1 K−1.

In [6]:
Model.diffusivity = 9e-7 * u.metre**2 / u.second 
Model.capacity    = 1000. * u.joule / (u.kelvin * u.kilogram)

## Air Layer

In [7]:
air = Model.add_material(name="Air", shape=GEO.shapes.Layer(top=Model.top, bottom=0 * u.kilometer))
air.density = 1. * u.kilogram / u.metre**3
air.diffusivity = 1e-6 * u.metre**2 / u.second
air.capacity = 1000. * u.joule / (u.kelvin * u.kilogram)


## Sedimentary Layers

In [8]:
Darai_vertices = [(21. * u.kilometer, -1.2 * u.kilometer),
                  (21. * u.kilometer, 2 * u.kilometer),
                  (3. * u.kilometer, 0. * u.kilometer),
                  (-16. * u.kilometer, 0. * u.kilometer),
                  (-21. * u.kilometer, 0.7 * u.kilometer),
                  (-21. * u.kilometer, -1.2 * u.kilometer)]

#Darai contains sediment wedges to influence the position of initial faults

Darai = Model.add_material(name="Darai_Limestone", shape=GEO.shapes.Polygon(Darai_vertices))
Darai.radiogenicHeatProd = 7.67e-7 * u.watt / u.meter**3
Darai.density  = 2700. * u.kilogram / u.metre**3

Ieru = Model.add_material(name="Ieru_Shale", shape=GEO.shapes.Layer2D(top=-1.2 * u.kilometer, bottom=-2. * u.kilometer))
Ieru.radiogenicHeatProd = 7.67e-7 * u.watt / u.meter**3
Ieru.density  = 2075. * u.kilogram / u.metre**3

Toro = Model.add_material(name="Toro_Sandstone", shape=GEO.shapes.Layer2D(top=-2. * u.kilometer, bottom=-2.5 * u.kilometer))
Toro.radiogenicHeatProd = 7.67e-7 * u.watt / u.meter**3
Toro.density  = 2600. * u.kilogram / u.metre**3

Imburu = Model.add_material(name="Imburu_Shale", shape=GEO.shapes.Layer2D(top=-2.5 * u.kilometer, bottom=-4. * u.kilometer))
Imburu.radiogenicHeatProd = 7.67e-7 * u.watt / u.meter**3
Imburu.density  = 2300. * u.kilogram / u.metre**3

## Basement

In [9]:
Basement = Model.add_material(name="Continental Crust", shape=GEO.shapes.Layer2D(top=-4. * u.kilometer, bottom=-7. * u.kilometer))
Basement.radiogenicHeatProd = 7.67e-7 * u.watt / u.meter**3
Basement.density  = 2720. * u.kilogram / u.metre**3

## Magobu Mudstone

In [10]:
magobu_vertices = [(-21. * u.kilometer, -5.5 * u.kilometer),
                   (-1. * u.kilometer, -5.5 * u.kilometer),
                   (0. * u.kilometer, -5. * u.kilometer),
                   (1. * u.kilometer, -4.2 * u.kilometer),
                   (1. * u.kilometer, -4. * u.kilometer),
                   (-21. * u.kilometer, -4. * u.kilometer),
                   (-21. * u.kilometer, -5.5 * u.kilometer)]

Magobu = Model.add_material(name="Magobu_Mudstone", shape=GEO.shapes.Polygon(magobu_vertices))
Magobu.radiogenicHeatProd = 7.67e-7 * u.watt / u.meter**3
Magobu.density  = 2350. * u.kilogram / u.metre**3

## Pre-defining Initial Faults and Decollements

In [11]:
Basal_Detachment = [(21. * u.kilometer, -4. * u.kilometer),
                         (1. * u.kilometer, -4. * u.kilometer),
                         (1. * u.kilometer, -3.3 * u.kilometer),
                         (21. * u.kilometer, -3.3 * u.kilometer),
                         (21. * u.kilometer, -4. * u.kilometer)]

Basal_Detachment = Model.add_material(name="Imburu_weak3", shape=GEO.shapes.Polygon(Basal_Detachment))
Basal_Detachment.radiogenicHeatProd = 7.67e-7 * u.watt / u.meter**3
Basal_Detachment.density  = 2300. * u.kilogram / u.metre**3


Graben_Fault = [(-4. * u.kilometer, -7. * u.kilometer),
                          (-3. * u.kilometer, -6.7 * u.kilometer),
                          (-2. * u.kilometer, -6.2 * u.kilometer),
                          (0. * u.kilometer, -5. * u.kilometer),
                          (1. * u.kilometer, -4.2 * u.kilometer),
                          (1. * u.kilometer, -4.0 * u.kilometer),
                          (0. * u.kilometer, -4.8 * u.kilometer),
                          (-1. * u.kilometer, -5.5 * u.kilometer),
                          (-2. * u.kilometer, -6. * u.kilometer),
                          (-3. * u.kilometer, -6.5 * u.kilometer),
                          (-4. * u.kilometer, -6.8 * u.kilometer)]

Graben_Fault = Model.add_material(name="Basement_weak", shape=GEO.shapes.Polygon(Graben_Fault))
Graben_Fault.radiogenicHeatProd = 7.67e-7 * u.watt / u.meter**3
Graben_Fault.density  = 2720. * u.kilogram / u.metre**3


## Rheology

In [12]:
rh = GEO.ViscousCreepRegistry()

In [13]:
Model.minViscosity = 5e18 * u.pascal * u.second
Model.maxViscosity = 5e23 * u.pascal * u.second

air.viscosity = 5e18 * u.pascal * u.second

Darai.viscosity = 1e22 * u.pascal * u.second

Ieru.viscosity = 8e19 * u.pascal * u.second

Toro.viscosity = 1e21 * u.pascal * u.second

Imburu.viscosity = 5e20 * u.pascal * u.second

Basement.viscosity = 1e23 * u.pascal * u.second

Magobu.viscosity = 5e20 * u.pascal * u.second

Basal_Detachment.viscosity = 1e19 * u.pascal * u.second

Graben_Fault.viscosity = 1e19 * u.pascal * u.second

In [14]:
#These layers are used for both the conveyor belt boundary condition, and the isostasy boundary condition. 

#The rigid base acts as a proxy for the mantle when le_code isostasy is used. Its density will tweak the magnitude of the flexural response.

fricLayerShape = GEO.shapes.Layer(top=Model.bottom + 4.6 * u.kilometer, bottom=Model.bottom + 4.1 * u.kilometer)
rigidBaseShape = GEO.shapes.Layer(top=Model.bottom + 4.1 * u.kilometer, bottom=Model.bottom)

rigidBase = Model.add_material(name="Frictional", shape=rigidBaseShape)
rigidBase.radiogenicHeatProd = 7.67e-7 * u.watt / u.meter**3
rigidBase.density  = 3500. * u.kilogram / u.metre**3
rigidBase.viscosity = 1e23 * u.pascal * u.second

frictionalBasal = Model.add_material(name="Frictional", shape=fricLayerShape)
frictionalBasal.viscosity = 1e23 * u.pascal * u.second
frictionalBasal.density = 3500.0 * u.kilogram / u.metre**3
frictionalBasal.plasticity = GEO.DruckerPrager(
                                               cohesion=0.1 * u.megapascal,
                                               frictionCoefficient=np.tan(np.radians(12.0)),
                                               frictionAfterSoftening=np.tan(np.radians(6.0)),
                                               epsilon1=0.01,
                                               epsilon2=0.06
                                               )

In [15]:
Model.init_model()

In [16]:
Darai.plasticity = GEO.DruckerPrager(name="Darai_Limestone",
                                     cohesion=5. * u.megapascal,
                                     cohesionAfterSoftening=0.5 * u.megapascal,
                                     frictionCoefficient=0.1,
                                     frictionAfterSoftening=0.01,
                                     epsilon1=0.1, epsilon2=0.25)
Ieru.plasticity = GEO.DruckerPrager(name="Ieru_Shale",
                                    cohesion=5. * u.megapascal,
                                    cohesionAfterSoftening=0.5 * u.megapascal,
                                    frictionCoefficient=0.1,
                                    frictionAfterSoftening=0.01,
                                    epsilon1=0.1, epsilon2=0.25)
Toro.plasticity = GEO.DruckerPrager(name="Toro_Sandstone",
                                    cohesion=5. * u.megapascal,
                                    cohesionAfterSoftening=0.5 * u.megapascal,
                                    frictionCoefficient=0.1,
                                    frictionAfterSoftening=0.01,
                                    epsilon1=0.1, epsilon2=0.25)
Imburu.plasticity = GEO.DruckerPrager(name="Imburu_Shale",
                                      cohesion=5. * u.megapascal,
                                      cohesionAfterSoftening=0.5 * u.megapascal,
                                      frictionCoefficient=0.1,
                                      frictionAfterSoftening=0.01,
                                      epsilon1=0.1, epsilon2=0.25)

Magobu.plasticity = GEO.DruckerPrager(name="Magobu_Mudstone",
                                      cohesion=5. * u.megapascal,
                                      cohesionAfterSoftening=0.5 * u.megapascal,
                                      frictionCoefficient=0.1,
                                      frictionAfterSoftening=0.01,
                                      epsilon1=0.1, epsilon2=0.25)

Basement.plasticity = GEO.DruckerPrager(name="Basement",
                                        cohesion=40. * u.megapascal,
                                        cohesionAfterSoftening=4. * u.megapascal,
                                        frictionCoefficient=0.6,
                                        frictionAfterSoftening=0.06,
                                        epsilon1=0.1, epsilon2=0.25)


Basal_Detachment.plasticity = GEO.DruckerPrager(name="Imburu_Fault",
                                            cohesion=1. * u.megapascal,
                                            cohesionAfterSoftening=0.01 * u.megapascal,
                                            frictionCoefficient=0.05,
                                            frictionAfterSoftening=0.01,
                                            epsilon1=0., epsilon2=0.01)

Graben_Fault.plasticity = GEO.DruckerPrager(name="Basement_Fault",
                                             cohesion=1. * u.megapascal,
                                             cohesionAfterSoftening=0.01 * u.megapascal,
                                             frictionCoefficient=0.05,
                                             frictionAfterSoftening=0.01,
                                             epsilon1=0., epsilon2=0.01)

## Heat Flow

In [17]:
Model.set_temperatureBCs(top=293.15 * u.degK, materials=[(air, 293.15*u.degK)])
Model.set_heatFlowBCs(bottom=(-0.044 * u.watt / u.metre**2, Basement))


## Velocity Boundary Conditions

In [18]:
import underworld.function as fn
velocity_fast = 2. * u.centimeter / u.year
velocity = 0.7 * u.centimeter / u.year

import underworld.function as fn

tapeL=frictionalBasal
flthick=GEO.nd(tapeL.top-tapeL.bottom)

conditions = [(Model.y <= GEO.nd(rigidBase.top), GEO.nd(-velocity)),
              (Model.y < GEO.nd(tapeL.top),
               GEO.nd(-velocity)*(flthick-(Model.y-GEO.nd(tapeL.bottom)))/flthick),
              (True, GEO.nd(0. * u.centimeter / u.year))]

fn_condition = fn.branching.conditional(conditions)

condition_right = [(Model.y > GEO.nd(-4. * u.kilometer), GEO.nd(-velocity_fast)),
                   (Model.y <= GEO.nd(-4. * u.kilometer), GEO.nd(-velocity))]
fn_condition_right = fn.branching.conditional(condition_right)

Model.set_velocityBCs(left=[fn_condition, None],
                      right=[fn_condition_right, 0.],
                      top=[None, None],
                      bottom=GEO.LecodeIsostasy(reference_mat=rigidBase, average=False))

In [19]:
Model.init_model()


## Interface Tracers

In [20]:
x = np.linspace(GEO.nd(Model.minCoord[0]), GEO.nd(Model.maxCoord[0]), 1000)
y = 0.

surface_tracers = Model.add_passive_tracers(name="Surface", vertices=[x,y])

## Grid Tracers

In [21]:
x_c, y_c = GEO.circles_grid(radius=0.1*u.kilometer, 
                     minCoord=[Model.minCoord[0], Basement.bottom], 
                     maxCoord=[Model.maxCoord[0], 0.*u.kilometer])

FSE_Crust = Model.add_passive_tracers(name="FSE_Crust", vertices=[x_c, y_c])

## Run Model

In [22]:
Model.init_model()

In [23]:
Model.run_for( 450001.* u.year, checkpoint_interval=5000. * u.year)


Running with UWGeodynamics version 2.8.5
Options:  -remove_constant_pressure_null_space False -ksp_type bsscr -restore_K False -ksp_k2_type NULL -change_backsolve False -rescale_equations False -Q22_pc_type uw -change_A11rhspresolve False -pc_type none -A11_ksp_rtol 1e-06 -A11_ksp_type fgmres -scr_ksp_rtol 1e-05 -scr_ksp_type fgmres
Step:     1 Model Time: 3961.1 year dt: 3961.1 year (2020-03-13 02:39:17)
Step:     2 Model Time: 5000.0 year dt: 1038.9 year (2020-03-13 02:39:46)
Step:     3 Model Time: 8961.1 year dt: 3961.1 year (2020-03-13 02:39:54)
Step:     4 Model Time: 10000.0 year dt: 1038.9 year (2020-03-13 02:40:25)
Step:     5 Model Time: 13961.1 year dt: 3961.1 year (2020-03-13 02:40:37)
Step:     6 Model Time: 15000.0 year dt: 1038.9 year (2020-03-13 02:41:07)
Step:     7 Model Time: 18961.1 year dt: 3961.1 year (2020-03-13 02:41:29)
Step:     8 Model Time: 20000.0 year dt: 1038.9 year (2020-03-13 02:42:17)
Step:     9 Model Time: 23961.1 year dt: 3961.1 year (2020-03-13 02:

Step:   105 Model Time: 263961.1 year dt: 3961.1 year (2020-03-16 00:06:10)
Step:   106 Model Time: 265000.0 year dt: 1038.9 year (2020-03-16 00:07:29)
Step:   107 Model Time: 268961.1 year dt: 3961.1 year (2020-03-16 00:08:27)
Step:   108 Model Time: 270000.0 year dt: 1038.9 year (2020-03-16 00:09:55)
Step:   109 Model Time: 273961.1 year dt: 3961.1 year (2020-03-16 00:10:55)
Step:   110 Model Time: 275000.0 year dt: 1038.9 year (2020-03-16 00:12:04)
Step:   111 Model Time: 278961.1 year dt: 3961.1 year (2020-03-16 00:12:49)
Step:   112 Model Time: 280000.0 year dt: 1038.9 year (2020-03-16 00:13:51)
Step:   113 Model Time: 283961.1 year dt: 3961.1 year (2020-03-16 00:14:31)
Step:   114 Model Time: 285000.0 year dt: 1038.9 year (2020-03-16 00:15:52)
Step:   115 Model Time: 288961.1 year dt: 3961.1 year (2020-03-16 00:16:56)
Step:   116 Model Time: 290000.0 year dt: 1038.9 year (2020-03-16 00:17:51)
Step:   117 Model Time: 293961.1 year dt: 3961.1 year (2020-03-16 00:18:21)
Step:   118 

/usr/local/lib/python3.5/dist-packages/UWGeodynamics/LecodeIsostasy/LecodeIsostasy.py:146: RuntimeWarning: divide by zero encountered in true_divide
  basal_velocities = -1.0 * botMeanDensities * sep_velocities_nodes / botMeanDensities0
/usr/local/underworld2/lib/underworld/systems/_bsscr.py:477: UserWarning: A PETSc error has been encountered during the solve. Solution fields are possibly erroneous. 

This error is probably due to an incorrectly constructed linear system. Please check that your boundary conditions are consistent and sufficient and that your viscosity is positive everywhere. If you are deforming the mesh, ensure that it has not become tangled. 

The resultant KSPConvergedReasons are (f_hat, outer, backsolve) (-9,-9,-9).


  warnings.warn(estring)
/usr/local/underworld2/lib/underworld/systems/_bsscr.py:496: UserWarning: A floating-point error has been detected during the solve. Solution fields are possibly erroneous. 

This is likely due to overly large value variations

RuntimeError: Error encountered. Full restart recommended as exception safety not guaranteed. Error message:
Error - in TimeIntegrand_SecondOrder(), for TimeIntegrand "1GP72UDN__integrand" of type SwarmAdvector: When trying to find time deriv for item 1456 in step 1, *failed*.

Unable to determine valid velocity for particle of local id 1456 with coordinate (0.433193,-0.265383) on process rank 0. Velocity appears infinite.